# Projet Knowledge Extraction - Partie A : Preprocessing et Représentation Text

## Étape 1 : Setup et Importations
Objectif : Configurer l'environnement et importer les librairies nécessaires.

In [ ]:
import sys
# Installation des dépendances si nécessaire
# !{sys.executable} -m pip install -r requirements.txt

import pandas as pd
import numpy as np
import nltk
import spacy
import preprocessing as pp # Notre module local

print("Environnement configuré avec succès.")